In [7]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift, fftfreq
import pandas as pd

**The next cell contains functions for analyzing data:**
- Load the csv of simulation data as a pandas dataframe. Creates arrays of the values in both axes.
- plot_data plots the simulation data with the first 200 points truncated, so that the data is fully comprised of a periodic signal.
- sub_mean removes the mean value from the truncated data set so that FFT gives the correct result.
- plot_FFT plots the FFT of the corrected data. "Freqs" is determined by the number of data samples and the time interval between each sample.
- m_average gives the moving box average of the dataset, which shows its low-frequency data trend. Changing the window size changes the frequency.
- find_time_lag uses cross-correlation to determine the lag between two signals. Compare the ideal heater (sin_heater) to the simulated data. 

In [64]:
#Load the simulation data:
#put quotes around the file name when loading!
df = pd.read_csv('fiber_point_monitor_30_sec.csv')
time_array = df['Physical Time (s)'].values
temp_array = df['fiber point 1 Monitor: fiber point 1 Monitor (K)'].values

# Plots simulation data, with the jump at the beginning truncated.
def plot_data(time_array, temp_array):
    plt.figure(figsize = (16, 6))
    plt.title('Temperature vs. Time at Fiber Point Monitor', fontsize = 14)
    plt.ylabel('Temperature [K]', fontsize = 12)
    plt.xlabel('Time [s]', fontsize = 12)
    plt.plot(time_array[200:], temp_array[200:], color = 'hotpink')
    
#Correct the temperature data by removing the mean and the jump:
def sub_mean(temp_array):
    temp_array = temp_array[200:]
    temp_sub_mean = temp_array - np.mean(temp_array)
    return temp_sub_mean

#Plot the FFT power spectrum of simulation data:
def plot_FFT(temp_array):
    fft_temp = np.fft.fft(sub_mean(temp_array))
    freqs = np.fft.fftfreq(len(fft_temp), 0.01)
    pow_temp = np.abs(fft_temp**2)
    plt.figure(figsize = (16, 6))
    plt.xlabel('Frequency')
    plt.ylabel('Temperature [arb.]')
    plt.title('FFT of temperature', fontsize = 14)
    plt.xlim(-2, 2)  #adjust or comment out for better visibility
    plt.plot(fftshift(freqs), fftshift(pow_temp))
    
#plot moving box average:
def m_average(data, window_size): #the frequency of the moving average should get smaller as the window size gets larger
    # makes sure that the sum of weights is equal to 1
    weights = np.repeat(1.0, window_size) / window_size #defines coefficients for moving avg calculation
    moving_avg = np.convolve(sub_mean(data), weights, 'valid')# Use np.convolve to compute the moving average
    plt.plot(moving_avg)

# Model the ideal sinusoidal heater which ignores amplitude arguments
t = np.linspace(0, 30, 3000) #timespan of the simulation in sec
sin_heater = np.sin(2*np.pi*(t+0.75))
    
#Use cross correlation to find time lag: 
def find_time_lag(signal1, signal2):
    cross_corr = np.correlate(signal1, signal2, mode='full')
    time_lag = np.argmax(cross_corr) - (len(signal1) - 1)
    print("Time lag:", time_lag, 'samples. The temperature measured at the fiber probe 
          lags the ideal heater by', time_lag * 0.01, 'seconds.')

In [66]:
find_time_lag(sin_heater, temp_array)

Time lag: 23 samples. The temperature measured at the fiber probe lags the ideal heater by 0.23 seconds.


^ check this for accuracy